In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import re


Setup Selenium WebDriver

In [ ]:
options = Options()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")


In [9]:
driver = webdriver.Edge(service=Service(r"C:\edgedriver_win64\msedgedriver.exe"), options=options)


Access URL

In [ ]:
base_url = "https://www.geonames.org/search.html?q=hotel&country=TN&startRow="
start_row = 0
all_data = []



# Function to clean latitude and longitude and scrap all the data 

In [ ]:

def clean_coordinates(coord):
    return re.sub(r'[^0-9\.\-]', ' ', coord).strip()

while True:
    url = base_url + str(start_row)
    driver.get(url)
    time.sleep(3)  
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.find("table", class_="restable")
    
    if table:
        rows = table.find_all("tr")[1:] 
        for row in rows:
            cols = row.find_all("td")
            if len(cols) >= 5:
                name = cols[1].text.strip()
                country = cols[2].text.strip()
                feature_class = cols[3].text.strip()
                latitude = clean_coordinates(cols[4].text.strip())
                longitude = clean_coordinates(cols[5].text.strip())
                all_data.append([name, country, feature_class, latitude, longitude])
    
    
    next_button = soup.find("a", string="next >")
    if next_button:
        start_row += 50  
    else:
        break  


driver.quit()



Save as DataFrame

In [ ]:
columns = ["Name", "Country", "Feature Class", "Latitude", "Longitude"]
df = pd.DataFrame(all_data, columns=columns)
df.to_csv("hotels_tunisia_cleaned.csv", index=False)